![](../img/330-banner.png)

Lecture 16: DBSCAN and Hierarchical Clustering
------------
UBC, 2022-23

Instructor: Amir Abdi

<br><br>

## Imports, announcements, LOs.

### Imports 

In [ ]:
import os
import random
import sys

import numpy as np
import pandas as pd

sys.path.append("../code/.")
from plotting_functions import *
from plotting_functions_unsup import *
import matplotlib.pyplot as plt
import mglearn
import seaborn as sns
from ipywidgets import interactive
from plotting_functions import *
from plotting_functions_unsup import *
from scipy.cluster.hierarchy import dendrogram, fcluster, linkage
from sklearn import cluster, datasets, metrics
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.datasets import make_blobs, make_moons
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import SilhouetteVisualizer

plt.rcParams["font.size"] = 16
%matplotlib inline
pd.set_option("display.max_colwidth", 0)

<br><br>

### Announcements

- HW6 is due Mar 15, 11:59pm
- Our final exam is going to Thu Apr 20 2023 | 19:00 pm
  - https://students.ubc.ca/enrolment/exams/exam-schedule

<br><br>

### Learning outcomes

From this lecture, students are expected to be able to:

- Identify **limitations of K-Means**.
- Broadly explain how **DBSCAN** works.
- Apply DBSCAN using `sklearn`. 
- Explain the effect of epsilon and minimum samples hyperparameters in DBSCAN.  
- Explain the difference between core points, border points, and noise points in the context of DBSCAN. 
- Identify DBSCAN limitations.
- Explain the idea of **hierarchical clustering**.
- Visualize dendrograms using `scipy.cluster.hierarchy.dendrogram`.    
- Explain the advantages and disadvantages of different clustering methods. 
- Apply clustering algorithms on image datasets and interpret clusters. 
- Recognize the impact of distance measure and representation in clustering methods. 

<br><br><br><br>

## Recap and motivation [[video](https://youtu.be/1ZwITQyWpkY)]

### K-Means recap 
- We discussed K-Means clustering in the previous lecture. 
- Each cluster is represented by a center. 
- Given a new point, you can assign it to a cluster by computing the distances to all cluster centers and **picking the cluster with the smallest distance.**
- It's a popular algorithm because 
    - It's easy to understand and implement.
    - Runs relatively quickly and scales well to large datasets. 
    - `sklearn` has a more scalable variant called [`MiniBatchKMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) which can handle very large datasets. 

### K-Means limitations

- Relies on **random initialization** and so the outcome may change depending upon this initialization. 
- K-Means clustering **requires to specify the number of clusters** in advance.
  - Very often you do not know the centers in advance. 
  - The elbow method or the silhouette method to find the optimal number of clusters are not always easy to interpret. 
- **All points will be assigned to a cluster**

#### Variants of K-Means to help mitigate the randomization effect

- **re-run the algorithm several times**. 
    - Check out `n_init` parameter of [sklearn's KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). 
    - **Best output will be chosen based on Inertia**
- [K-Means++](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf)
    - it picks the initial centroids which are far away from each other. 
    - In other words, K-Means++ gives more chance to select points that are far away from centroids already picked.    
    - By default `sklearn` uses this strategy for initialization. 

<br><br><br><br><br><br><br>

### K-Means limitations: Shape of K-Means clusters

- K-Means partitions the space based on the closest mean. 
- Each cluster is defined solely by its center and so it can only capture relatively simple shapes. 
- So the boundaries between clusters are **linear**; It fails to identify **clusters with complex shapes**. 
[Source](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html).

![](../img/kmeans_boundaries.png)



### K-Means: failure case 1

- K-Means performs poorly if the clusters have more complex shapes (e.g., two moons data below). 

In [ ]:
X, y = make_moons(n_samples=200, noise=0.05, random_state=42)
plot_X_k_means(X, k=2)

### K-Means: failure case 2

- It assumes that all directions are equally important for each cluster
  - Fails to identify non-spherical clusters. 

In [ ]:
# generate some random cluster data
X, y = make_blobs(random_state=170, n_samples=200)
rng = np.random.RandomState(74)
transformation = rng.normal(size=(2, 2))
X = np.dot(X, transformation)
plot_X_k_means(X, 3)

### K-Means: failure case 3 

- Again, K-Means is unable to capture complex cluster shapes. 

In [ ]:
X = datasets.make_circles(n_samples=200, noise=0.06, factor=0.4)[0]
plot_X_k_means(X, 2)

- Can we do better than this? 
- Another clustering algorithm called DBSCAN is able to tackle some of these cases. 

<br><br><br><br><br><br>

## DBSCAN [[video](https://youtu.be/T4NLsrUaRtg)]

- **D**ensity-**B**ased **S**patial **C**lustering of **A**pplications with **N**oise

### DBSCAN introduction

- DBSCAN is a **density-based** clustering algorithm. 
- Intuitively, it's based on the idea that clusters form dense regions in the data and so it works by identifying "crowded" regions in the feature space. 
- It can address some of the limitations of K-Means we saw above. 
    - It does not require the user to specify the number of clusters in advance. 
    - It can identify points that are not part of any clusters. 
    - It can capture clusters of complex shapes. 

Let's try `sklearn`'s DBSCAN.   

In [ ]:
X, y = make_moons(n_samples=200, noise=0.08, random_state=42)

# ------ new code ------------
dbscan = DBSCAN(eps=0.2)
dbscan.fit(X)
# -----------------------

plot_X_dbscan(X, dbscan)

In [ ]:
dbscan.labels_

- DBSCAN is able to capture half moons shape
- **We don't not have to specify the number of clusters**. 
    - That said, it has two other non-trivial hyperparameters to tune. 
      - eps
      - min_samples
- There are two examples which have **not been assigned any label** (noise examples). 

One more example of DBSCAN clusters capturing complex cluster shapes. 

In [ ]:
X = datasets.make_circles(n_samples=200, noise=0.06, factor=0.4)[0]
dbscan = DBSCAN(eps=0.3, min_samples=3)
dbscan.fit(X)
plot_X_dbscan(X, dbscan)

### How does it work?

- Iterative algorithm.  
- Based on the idea that clusters form dense regions in the data. 

![](../img/DBSCAN_search.gif)

<!-- <center>
<img src="img/DBSCAN_search.gif" alt="" height="900" width="900"> 
</center>
 -->
[Source](https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/)

### Two main hyperparameters
- `eps`: determines what it means for points to be "close"
- `min_samples`: determines the number of **neighboring points** we require to consider in order for a point to be part of a cluster

### Effect of `eps` hyperparameter

- `eps`: determines what it means for points to be "close"

In [ ]:
X, y = make_blobs(random_state=0, n_samples=11)

In [ ]:
interactive(lambda eps=0.1: plot_dbscan_with_labels(X, eps, min_samples=3), eps=(0.1, 6, 0.5))

In [ ]:
pd.DataFrame(euclidean_distances(X, X))

### Effect of `min_samples` hyperparameter

In [ ]:
interactive(
    lambda min_samples=1: plot_dbscan_with_labels(X, eps=1.2, min_samples=min_samples),
    min_samples=(1, 5, 1),
)

### More details on DBSCAN

There are three kinds of points.

- **Core points** are the points that have at least `min_samples` points within a distance of `eps`

- **Border points** are connected to a core point. They are within a distance of eps to core point but they have fewer than `min_samples` points within a distance of `eps`. 

- **Noise points** are the points which do not belong to any cluster. In other words, the points which have less than `min_samples` points within distance `eps` of the starting point are noise points. 


### DBSCAN algorithm 

- Pick a point $p$ at random.
- Check whether $p$ is a "core" point or not (if not, pick another point). 
  - You can check this by looking at the number of neighbours within **epsilon distance** if they have at least `min_samples` points in the neighbourhood
- If $p$ is a core point, give it a colour (label). 
  - Spread the colour of $p$ to all of its neighbours.
- Check if **any of the neighbours that received the colour is a core point**, 
  - if yes, spread the colour to its neighbors as well.
- Once there are no more core points left to spread the colour, pick a new unlabeled point $p$ and repeat the process.

### K-Means vs. DBSCAN

- In DBSCAN, you do not have to specify the number of clusters! 
    - Instead, you have to tune `eps` and `min_samples`. 
- Unlike K-Means, DBSCAN doesn't have to assign all points to clusters. 
    - The label is -1 if a point is unassigned.
- In sklearn, there is no `predict` method for DBSCAN.
    - DBSCAN only really clusters the points you have, not "new" or "test" points.
    - (you can still do whatever you like with the clsuter)

### Illustration of hyperparameters `eps` and `min_samples`

In [ ]:
X, y = make_blobs(random_state=0, n_samples=12)
mglearn.discrete_scatter(X[:, 0], X[:, 1]);

In [ ]:
dbscan = DBSCAN()
clusters = dbscan.fit_predict(X)
print("Cluster memberships:{}".format(clusters))

- Default values for hyperparameters don't work well on toy datasets. 
- All points have been marked as noise with the default values for `eps` and `min_samples`
- Let's examine the **effect of changing these hyperparameters**. 
    - noise points: shown in white
    - core points: bigger
    - border points: smaller

**Homework**: play with min_samples and eps in the above example and find a combination that results in 2 clusters.

<br><br><br><br><br><br>


In [ ]:
mglearn.plots.plot_dbscan()

### Observations

- Increasing `eps` ($\uparrow$) (left to right in the plot above) means more points will be included in a cluster. 
    - `eps` = 1.0 either creates more clusters or more noise points, whereas eps=3.0 puts all points in one cluster with no noise points.  
- Increasing `min_samples` ($\uparrow$) (top to bottom in the plot above) means points in less dense regions will be labeled as noise. 
    - `min_samples=2`, for instance, has none or only a fewer noise points whereas `min_samples=5` has several noise points. 
- Here `min_samples` = 2.0 or 3.0 and `eps` = 1.5 is giving us the best results. 
- In general, it's not trivial to tune these hyperparameters. 


### Question for you

- Does the order that you pick the points matter in DBSCAN?
<br><br><br><br>

**Yes and No**. 
- Any of the cluster's core points is able to fully identify the cluster, with no randomness involved. 
- But, there is a possible conflict you might get is that if a border point is reachable from two clusters. In this case the assignment will be implementation dependent, but *usually* the border point will be assigned to the **first cluster that "finds" it**. 

### Evaluating DBSCAN clusters 
We can use the silhouette method because it's not dependent on the idea of cluster centers. 

In [ ]:
X, y = make_blobs(random_state=100, centers=3, n_samples=300)
dbscan = DBSCAN(eps=2, min_samples=5)
dbscan.fit(X)
plot_X_dbscan(X, dbscan)

In [ ]:
# Yellowbrick is designed to work with K-Means and not with DBSCAN.
# So it needs the number of clusters stored in n_clusters
# It also needs `predict` method to be implemented.
# So we are patching the object with the right attributes here so that we can use Yellowbrick to show Silhouette plots.
# python is cool, eh?

n_clusters = len(set(dbscan.labels_))
dbscan.n_clusters = n_clusters
dbscan.predict = lambda x: dbscan.labels_

In [ ]:
visualizer = SilhouetteVisualizer(dbscan, colors="yellowbrick")
visualizer.fit(X)  # Fit the data to the visualizer
visualizer.show();

### Summary: Pros and cons

- Pros
    - Can learn arbitrary cluster shapes
    - Can detect outliers 
- Cons
    - sklearn does not perform `predict` on new examples (but you can still use clusters for prediction if you like)
    - Needs tuning of two non-obvious hyperparameters 

There is an improved version of DBSCAN called [`HDBSCAN` (hierarchical DBSCAN)](https://github.com/scikit-learn-contrib/hdbscan). 

### DBSCAN: failure cases

- DBSCAN is able to capture complex clusters. But this doesn't mean that `DBSCAN` always works better. It has its own problems! 
- DBSCAN doesn't do well when we have **clusters with different densities**. 
    - You can play with the hyperparameters but it's not likely to help much.

### DBSCAN: failure cases

- Let's consider this dataset with three clusters of varying densities.  
- K-Means performs better compared to DBSCAN. But it has the benefit of knowing the value of $K$ in advance. 


In [ ]:
X_varied, y_varied = make_blobs(
    n_samples=200, cluster_std=[1.0, 5.0, 1.0], random_state=10
)
plot_k_means_dbscan_comparison(X_varied)

### (iClicker) Exercise 16.1 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) With tiny epsilon (`eps` in `sklearn`) and min samples=1 (`min_samples=1` in `sklearn`) we are likely to end up with each point in its own cluster. 
- (B) With a smaller value of `eps` and larger number for `min_samples` we are likely to end up with a one big cluster. 
- (C) K-Means is more susceptible to outliers compared to DBSCAN.  
- (D) In DBSCAN to be part of a cluster, each point must have at least `min_samples` neighbours in a given radius (including itself). 
- (E) In DBSCAN, it is generally a good idea to run DBSCAN with a large number of different random orderings of training examples. 

<br><br><br><br>

<br><br><br><br>

## Hierarchical clustering [[video]()]

### Motivation

- Deciding how many clusters we want is a hard problem. 
- Often, it's useful to get a complete picture of similarity between points in our data before picking the number of clusters.  
- Hierarchical clustering is helpful in these scenarios.  

### Main idea

1. Start with each point in its own cluster. 
2. Greedily merge most similar *clusters*. 
3. Repeat Step 2 until you obtain only one cluster ($n-1$ times).

### Visualizing hierarchical clustering

- Hierarchical clustering can be visualized using a tool called **a dendrogram**. 
- Unfortunately, `sklearn` cannot do it so we will use the package `scipy.cluster.hierarchy` for hierarchical clustering.

### Hierarchical clustering input and output

In [ ]:
from scipy.cluster.hierarchy import ward
X, y = make_blobs(random_state=0, n_samples=11)
linkage_array = ward(X)

In [ ]:
plot_X_dendrogram(X, linkage_array, label_n_clusters=True)

- Every point goes through the journey of being on its own (its own cluster) and getting merged with some other bigger clusters. 
- The intermediate steps in the process provide us clustering with different number of clusters. 

### Dendrogram

- Dendrogram is a tree-like plot. 
- On the x-axis we have data points. 
- On the y-axis we have **distances between clusters**. 
- We start with data points as leaves of the tree.  
- New parent node is created for every two clusters that are joined. 
- The length of each branch shows how far the merged clusters go. 
    - In the dendrogram above going from three clusters to two clusters means merging far apart points because the branches between three cluster to two clusters are long. 

### How to plot a dendrogram? 

In [ ]:
from scipy.cluster.hierarchy import dendrogram

ax = plt.gca()
dendrogram(linkage_array, ax=ax)
plt.xlabel("Sample index")
plt.ylabel("Cluster distance");

### What do we mean by distance between clusters?

- We know how to measure distance between points (e.g., using Euclidean distance). 
- How do we measure distances between clusters? 
- The **linkage criteria** determines how to find similarity between clusters:
- Some example linkage criteria are: 
    - single linkage (minimum distance)
    - average linkage (average distance)
    - complete (or maximum) linkage (maximum distance)
    - ward linkage    

In [ ]:
plot_X_dendrogram(X, linkage_array, label_n_clusters=True)

### `single` linkage 

- **Merges two clusters that have the smallest *minimum* distance across all their points**
- Let's use `scipy.cluster.hierarchy`'s `single` to get linkage information. 
- This method gives us matrix `Z` with the merging information. 

In [ ]:
from scipy.cluster.hierarchy import (
    average,
    complete,
    dendrogram,
    fcluster,
    single,
    ward,
)

Z = single(X)
columns = ["c1", "c2", "distance(c1, c2)", "# observations"]

In [ ]:
X.shape

In [ ]:
plt.scatter(X[:, 0], X[:, 1])

In [ ]:
pd.DataFrame(Z, columns=columns)

- The linkage returns a matrix `Z` of shape `n-1` (number of iterations) by 4:
  - The rows represent iterations. 
  - First and second columns (c1 and c2 above): indexes of the clusters being merged.
  - Third column (distance(c1, c2)): the distance between the clusters being merged.
  - Fourth column (# observations): the number of examples in the newly formed cluster.

### Creating dendrogram with `single` linkage

In [ ]:
dendrogram(Z)
# Z is our single linkage matrix
plt.title("Dendrogram (Single linkage)");

### `average` linkage 

- **Merges two clusters that have the smallest average distance between all their points.**
- `scipy.cluster.hierarchy`'s `average` method gives us matrix `Z` with the merging information using average linkage.   

In [ ]:
Z = average(X)
dendrogram(Z)
# Dendrogram with average linkage
plt.title("Dendrogram (Average linkage)");

### `complete` linkage

- Merges two clusters that have the **smallest maximum distance between their points**. 

In [ ]:
Z = complete(X)
dendrogram(Z)
# Dendrogram with complete linkage
plt.title("Dendrogram (Complete linkage)");

### `ward` linkage

- Picks two clusters to merge such that the **variance within all clusters increases the least**. 
- Often leads to equally sized clusters. 

In [ ]:
Z = ward(X)
dendrogram(Z)
# Dendrogram with ward linkage
plt.title("Dendrogram (Ward linkage)");

<br><br>

### Hierarchical clustering on UN Subvotes dataset

- Let's use a dataset of votes on UN resolutions:

In [ ]:
votes_df = pd.read_csv("../data/subvotes.csv")
votes = votes_df.pivot(index="country", columns="rcid")
votes = votes[np.sum(np.isnan(votes), axis=1) < 1]
print(votes.shape)
votes

- We have 17 countries and 368 votes. 
- Let's cluster countries based on how they vote. 

- We'll use [hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) here because we are interested in knowing whether the countries agreed or disagreed on resolutions. 

In [ ]:
def plot_dendrogram(method="average", metric="hamming", w=22, h=12):
    Z = linkage(votes, method=method, metric=metric)
    fig, ax = plt.subplots(figsize=(w, h))
    dendrogram(Z, labels=votes.index, ax=ax)
    ax = plt.gca()
    ax.set_ylabel("Distance", fontsize=w)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=80, fontsize=w)

In [ ]:
plot_dendrogram()

### (Optional) Truncation
- If you want to truncate the tree in the dendrogram (specially when you have a big $n$) you can use the `truncate_mode`. 

In [ ]:
Z = linkage(votes, method="average", metric="hamming")
fig, ax = plt.subplots(figsize=(18, 10))
dendrogram(Z, p=6, truncate_mode="lastp", ax=ax, labels=votes.index);
# p is the number of leaves when truncate mode is "lastp"

- Alternatively, you can truncate the tree down to $x$ levels from the single cluster:

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))
dendrogram(Z, p=6, truncate_mode="level", ax=ax, labels=votes.index);
# p is the max depth of the tree when truncate_mode is "level"

Let's go back to our toy dataset to understand truncation better. 

In [ ]:
ax = plt.gca()
dendrogram(linkage_array, ax=ax)
plt.xlabel("Sample index")
plt.ylabel("Cluster distance");

In [ ]:
dendrogram(Z, p=3, truncate_mode="level");
# p is the max depth of the tree

In [ ]:
dendrogram(Z, p=4, truncate_mode="lastp");
# p is the number of leaf nodes

### Flat cluster 

- To bring the clustering to a "flat" format, we can use `fcluster`

In [ ]:
from scipy.cluster.hierarchy import fcluster

cluster_labels = fcluster(Z, 6, criterion="maxclust")

In [ ]:
pd.DataFrame(cluster_labels, votes.index)

To understand this better, let's try it out on our toy dataset. 

In [ ]:
ax = plt.gca()
dendrogram(linkage_array, ax=ax)
plt.xlabel("Sample index")
plt.ylabel("Cluster distance");

In [ ]:
cluster_labels = fcluster(linkage_array, 3, criterion="maxclust")
pd.DataFrame(cluster_labels, columns=["Cluster"])

Question: If we increase number of clusters to 4, what are the members of the newly formed 4th cluster?

Answer: ???

<br><br>

## ❓❓ Questions for you

### (iClicker) Exercise 16.2

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) In hierarchical clustering we do not have to worry about initialization. 
- (B) Hierarchical clustering can only be applied to smaller datasets because dendrograms are hard to visualize for large datasets.
- (C) In all the three clustering methods we saw (K-Means, DBSCAN, hierarchical clustering), there is a way to decide the granularity of clustering (i.e., how many clusters to pick). 
- (D) To get robust clustering we can naively ensemble cluster labels (e.g., pick the most popular label) produced by different clustering methods. 
- (E) If you have a high Silhouette score and very clean and robust clusters, it means that the algorithm has captured the semantic meaning in the data of our interest.   
<br><br><br><br>

## Applying clustering on face images 

- We'll be working with `sklearn`'s [Labeled Faces in the Wild dataset](https://scikit-learn.org/0.16/datasets/labeled_faces.html). 
- The dataset has images of celebrities from the early 2000s downloaded from the internet. 

> Credit: This example is based on the example from [here](https://learning.oreilly.com/library/view/introduction-to-machine/9781449369880/ch03.html).

In [ ]:
import matplotlib as mpl
from sklearn.datasets import fetch_lfw_people

mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["image.cmap"] = "gray"

### Example images from the dataset

In [ ]:
people = fetch_lfw_people(min_faces_per_person=20, resize=0.7)

fig, axes = plt.subplots(2, 5, figsize=(10, 5), subplot_kw={"xticks": (), "yticks": ()})
for target, image, ax in zip(people.target, people.images, axes.ravel()):
    ax.imshow(image)
    ax.set_title(people.target_names[target])

In [ ]:
plt.show()

In [ ]:
image_shape = people.images[0].shape
print("people.images.shape: {}".format(people.images.shape))
print("Number of classes: {}".format(len(people.target_names)))

There are 3,023 images stored as arrays of 5655 pixels (87 by 65), of 62 different people:

In [ ]:
counts = np.bincount(people.target)  # count how often each target appears
df = pd.DataFrame(counts, columns=["count"], index=people.target_names)
df.sort_values("count", ascending=False)

Let's make the data less skewed by taking only 20 images of the each person. 

In [ ]:
mask = np.zeros(people.target.shape, dtype=bool)
for target in np.unique(people.target):
    mask[np.where(people.target == target)[0][:20]] = 1

X_people = people.data[mask]
y_people = people.target[mask]

In [ ]:
X_people.shape

<br><br><br><br><br>

----------------

**[Study on your own]**

### Representation of images (Optional)

- Representation of input is very important when you cluster examples. 
- In this example, we'll use **Principal Component Analysis (PCA) representation** (briefly talked about last session; not covered in this course)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100, whiten=True, random_state=0)
X_pca = pca.fit_transform(X_people)
X_pca.shape

### Clustering faces with K-Means

We'll cluster the images with this new representation. 

In [ ]:
km = KMeans(n_clusters=10, random_state=10)
km.fit(X_pca)

What are the sizes of the clusters? 

In [ ]:
labels_km = km.fit_predict(X_pca)
print("Cluster sizes k-means: {}".format(np.bincount(labels_km)))

Let's examine cluster centers. Are they going to be real images from the dataset?

In [ ]:
def get_cluster_images(km, X_people, y_people, target_names, X_pca=None, pca=None, cluster=0):
    image_shape = (87, 65)
    fig, axes = plt.subplots(1, 6, subplot_kw={'xticks': (), 'yticks': ()},
                             figsize=(10, 10), gridspec_kw={"hspace": .3})
    center = km.cluster_centers_[cluster]
    
    mask = km.labels_ == cluster
    
    if pca: 
        dists = np.sum((X_pca - center) ** 2, axis=1)
        dists[~mask] = np.inf
        inds = np.argsort(dists)[:5]
        axes[0].imshow(pca.inverse_transform(center).reshape(image_shape), vmin=0, vmax=1)        
    else: 
        dists = np.sum((X_people - center) ** 2, axis=1)
        dists[~mask] = np.inf
        inds = np.argsort(dists)[:5]
        axes[0].imshow(center.reshape(image_shape), vmin=0, vmax=1)        
        
    axes[0].set_title('Cluster center %d'%(cluster))
    i = 1
    for image, label in zip(X_people[inds], y_people[inds]):
        axes[i].imshow(image.reshape(image_shape), vmin=0, vmax=1)
        axes[i].set_title("%s" % (target_names[label].split()[-1]), fontdict={'fontsize': 9})    
        i+=1
    plt.show()

In [ ]:
plot_faces_cluster_centers(km, pca)

- The centers found by K-Means are smooth versions of faces which makes sense. 
- Intuitively, they seem to capture some interesting characteristics of faces

Let's examine images for different centers. 

In [ ]:
get_cluster_images(
    km, X_people, y_people, people.target_names, pca=pca, X_pca=X_pca, cluster=3
)

In [ ]:
get_cluster_images(
    km, X_people, y_people, people.target_names, pca=pca, X_pca=X_pca, cluster=8
)

<br><br>

### Clustering faces with DBSCAN

In [ ]:
dbscan = DBSCAN()
labels = dbscan.fit_predict(X_pca)
print("Unique labels: {}".format(np.unique(labels)))

With default hyperparameters, we get all points as noise points. 

#### Tuning `eps`

In [ ]:
people.target_names[y_people].shape

Let's examine at distances between images.  

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

dists = euclidean_distances(X_pca)
np.fill_diagonal(dists, np.inf)

dist_df = pd.DataFrame(
    dists, index=people.target_names[y_people], columns=people.target_names[y_people]
)

dist_df.iloc[10:20, 10:20]

In [ ]:
dist_df.describe()

In [ ]:
labels

In [ ]:
for eps in [6, 7, 8, 9, 10, 11, 12, 14]:
    print("\neps={}".format(eps))
    dbscan = DBSCAN(eps=eps, min_samples=3)
    labels = dbscan.fit_predict(X_pca)
    print("Number of clusters: {}".format(len(np.unique(labels)) - 1))
    print("Cluster sizes: {}".format(np.bincount(labels + 1)))

- For lower `eps` all images are labeled as noise.
- For `eps=7` we get many noise points and a small cluster.
- For `eps=8` and `eps=9` we get many noise points but we also get one large cluster and a one smaller cluster.
- Starting `eps=10` we get one big cluster and noise points. 
- There is never more than one large cluster suggesting that all the images are more or less equally similar/dissimilar to the rest. 

#### Noise images identified by DBSCAN

In [ ]:
dbscan = DBSCAN(eps=14, min_samples=3)
labels = dbscan.fit_predict(X_pca)
print("Unique labels: {}".format(np.unique(labels)))

In [ ]:
print_dbscan_noise_images(X_people, y_people, dbscan, labels)

In [ ]:
plt.show()

- We can guess why these images are noise images. There are odd angles, cropping, sun glasses, hands near faces etc. 

Let's examine DBSCAN clusters. 

In [ ]:
dbscan = DBSCAN(min_samples=3, eps=8)
labels = dbscan.fit_predict(X_pca)
print("Number of clusters: {}".format(len(np.unique(labels))))
print("Cluster sizes: {}".format(np.bincount(labels + 1)))

- Some clusters correspond to people with distinct faces and facial expressions. 
- It's also capturing orientation of the face.  

In [ ]:
print_dbscan_clusters(X_people, y_people, labels)

In [ ]:
plt.show()

<br><br>

### (Optional) Clustering faces with hierarchical clustering 

Let's examine the dendrogram.  

In [ ]:
Z = ward(X_pca)
plt.figure(figsize=(20, 15))
dendrogram(Z, p=7, truncate_mode="level", no_labels=True)
plt.xlabel("Sample index")
plt.ylabel("Cluster distance");

In [ ]:
cluster_labels = fcluster(Z, 40, criterion="maxclust")  # let's get flat clusters

In [ ]:
from plotting_functions_unsup import print_hierarchical_clusters
hand_picked_clusters = [2, 3, 6, 29, 30, 36, 38]
print_hierarchical_clusters(
    X_people, y_people, people.target_names, cluster_labels, hand_picked_clusters
)
plt.show()

[End of study on your own]

----------

<br><br><br><br>

## Final comments, summary, and reflection

### Take-home message

- We saw three methods for clustering: K-Means, DBSCAN, and hierarchical clustering. 
- There are many more clustering algorithms out there which we didn't talk about. For example see [this overview of clustering methods](https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods). 
- Two important aspects of clustering
    - Choice of **distance metric**
    - Data representation
- Choosing the appropriate hyper-parameters (number of clusters, epsilon, etc.) for a given problem is quite hard. 
- A lot of manual interpretation is involved in clustering. 

### A few comments on clustering evaluation

- If you know the ground truth, you can use metrics such as [adjusted random score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html) or [normalized mutual information score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html). 
- We can't use accuracy scores. 
    - Because the labels themselves are meaningless in clustering.  
- Usually labels are not available, and if it is available we would probably go with **supervised models**.     


- The silhouette score works for different clustering methods and it can give us some intuition about the quality of clusters. But it's not very interpretable on real-world datasets.

A couple of ways to evaluate clustering: 
- Using *robustness-based* clustering metrics
- The idea is to run a clustering algorithm (or a number of clustering algorithms) after **adding some noise to the data** or **using different parameter settings** and comparing outcomes. 
- If many models, perturbations, and parameters are giving the same result, the clustering is likely to be trustworthy.  


**Even though all clustering models give similar results, the clusters might not capture the aspect you are interested in.**

<br><br>

## Resources 

- Check out this nice comparison of [sklearn clustering algorithms](https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods).
- [DBSCAN Visualization](https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/)
- [Clustering with Scikit with GIFs](https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/)